In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile, os, urllib.request, glob, math, shutil, sys
import torchvision
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, precision_recall_fscore_support, roc_auc_score, accuracy_score
import sklearn.metrics
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import PIL
import PIL.Image

sys.path.append('drive/MyDrive')
import modelArchs

fold_number = 4
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive


In [ ]:
train_dataset = pd.read_csv('drive/MyDrive/train_dataset_app_Fold_' + str(fold_number) + '.csv')
test_dataset = pd.read_csv('drive/MyDrive/test_dataset_app_Fold_' + str(fold_number) + '.csv')
print(len(train_dataset))
print(len(test_dataset))



Οι εικόνες είναι αποθηκευμένες στο drive σε zip.

Υπάρχουν 16 zip files, ένα για κάθε κλάση από το train και test dataset.

Κάνω extract τις εικόνες.


In [ ]:
os.makedirs('train_imgs')
os.makedirs('test_imgs')

prefix = 'drive/MyDrive/'

for i in ['train_imgs', 'test_imgs']:
  for j in ['ACK', 'BCC', 'MEL', 'NEV', 'SCC', 'SEK']:

    name = i + '_' + j
    src_zip = prefix + name + '.zip'
    dest_zip = i + '/' + j

    with zipfile.ZipFile(src_zip, 'r') as zip_ref:
      zip_ref.extractall(dest_zip)

    print("Finished with: ", name)

Κάνω load το εκάστοτε pretrained μοντέλο (MobileNet_V3_Large , Efficient Net,Resnet34, GoogleNet, MobileNet_V3_Large + SA(https://pytorch.org/vision/stable/models.html#classification), trained πάνω στο imagenet. Στο τελευταίο layer αλλάζουμε τα out_features από 1000 σε 6, καθώς έχουμε 6 διαφορετικές κλάσεις προς ταξινόμηση.

In [ ]:
# + ta ypoloipa montela, apla exo afisei mono to mobilenet + sa
model = modelArchs.get_MobileNetSa(6,True)
model.to(device)

Ορίζω διάφορα transformations για να κάνουμε augment το training set, ώστε να πετύχουμε καλύτερο generalization του μοντέλου.

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomResizedCrop(size = 224, scale = (0.4,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness = [0.7, 1.3], contrast = [0.7, 1.3], saturation = [0.7, 1.3], hue = [-0.1,0.1]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transforms_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Ορισμός υπερ-παραμέτρων.

In [ ]:
epochs = 30
batch_len = 32

criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma=0.2)

Εδώ ορίζω τα dataset και dataloaders, χρησιμοποιώντας το weightRandomSampler για να δημιουργήσει balanced batches:

https://towardsdatascience.com/demystifying-pytorchs-weightedrandomsampler-by-example-a68aceccb452

In [ ]:
def get_weights_for_imgs(images, n_categories):

    n_images = len(images)

    imgs_per_category = [0] * n_categories
    for _, image_category in images:
      imgs_per_category[image_category] += 1

    weight_per_category = [0] * n_categories # έχουμε 6 κατηγορίες
    for i in range(n_categories):
      weight_per_category[i] = 1 / float(imgs_per_category[i])#float(n_images) / float(imgs_per_category[i])

    weights = [0] * n_images
    for idx, (_, image_category) in enumerate(images):
        weights[idx] = weight_per_category[image_category]

    return weights

train_dataset = datasets.ImageFolder('train_imgs', transforms_train)
test_dataset = datasets.ImageFolder('test_imgs', transforms_test)

len_train_dataset = len(train_dataset)
len_test_dataset = len(test_dataset)

print("Length of train_dataset: ", len_train_dataset)
print("Length of test dataset: ", len_test_dataset)

weights = get_weights_for_imgs(train_dataset.imgs, len(train_dataset.classes))
weights = torch.DoubleTensor(weights)
train_sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_len, sampler = train_sampler, pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_len, shuffle=False)

['ACK', 'BCC', 'MEL', 'NEV', 'SCC', 'SEK']

Ορίζω συναρτήσεις για υπολογισμό του accuracy και του confusion matrix πάνω στο test set. Στη συνέχεια θα δημιουργήσω και άλλες συναρτήσεις για υπολογισμό F1-score, sensitivity, specificity, precision.

In [ ]:
def test_model(model,test_dataloader):

  pred_list = []
  label_list = []
  output_probs = []
  metrics = []

  for i, (inputs, labels) in enumerate(test_dataloader,1):

    label_list.extend(labels.data.tolist())

    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    probs = torch.softmax(outputs.data.cpu(), dim = 1).tolist()
    output_probs.extend(probs)

    pred =  torch.max(outputs, 1)[1].data.cpu().tolist()
    pred_list.extend(pred)

  conf_matrix = confusion_matrix(label_list , pred_list)

  metrics.append(accuracy_score(label_list, pred_list))
  metrics.extend(precision_recall_fscore_support(label_list,pred_list, average = 'macro'))
  metrics.append(roc_auc_score(label_list,output_probs, average = 'macro', multi_class = 'ovr'))
  metrics.extend(precision_recall_fscore_support(label_list,pred_list, average = 'weighted'))
  metrics.append(roc_auc_score(label_list,output_probs, average = 'weighted', multi_class = 'ovr'))

  return metrics, conf_matrix

Το βασικό train loop.

In [ ]:
for epoch in range(1, epochs + 1):

  model.train()
  train_running_loss = 0

  #Training phase
  print("Starting epoch: " , epoch)

  for i, (inputs, labels) in enumerate(train_dataloader,1):
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      train_running_loss += loss.detach().item()

  print('Epoch: %d | Loss: %.4f ' %(epoch, train_running_loss / i))
  scheduler.step()

  model.eval()
  try:
    metrics,conf_matrix = test_model(model,test_dataloader)
    print('Test metrics on epoch %d:' %(epoch))
    print(metrics)
    print("Confusion matrix: ")
    print(conf_matrix)
  except:
    print("Error in calculating metrics")

  try:
    checkpoint = {'model': model.state_dict(), 'epoch' : epoch}
    checkpoint_fn = 'drive/MyDrive/checkpoint' + str(epoch) + '.pt'
    torch.save(checkpoint, checkpoint_fn)
  except:
    print("Error in writing general checkpoint")

Ένα μικρό τέστ για να δώ ότι ο dataloader παράγει σωστά τα shapes των tensors, και ότι κάθε batch είναι ισορροπημένο. Το τεστ έγινε πριν το train.

In [ ]:
from collections import Counter

for inputs, labels in train_dataloader:
  inputs = inputs.to(device)
  print("Input size:", inputs.shape)
  out = model(inputs)
  print("Output size: ", out.shape)

  labels = labels.tolist()
  print("Test if batch is balanced: ")
  print(Counter(labels).keys())
  print(Counter(labels).values())
  break